In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import csv
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import classification_report

In [2]:
raw_primary = []
raw_secondary = []
#lines=393732;
lines=10000;
#df=pd.read_csv('PDB_CLEAN_DATA.csv')
#display(df)
with open('D:\Study\Thesis\Q8\PDB_CLEAN_DATA.csv') as csv_file:
	csv_reader = csv.reader(csv_file, delimiter=',')
	line_count = 0
	for row in csv_reader:
		add = True
		if line_count == 0:
			line_count += 1
		elif line_count > lines:
			break;
		else:
			for j in range(len(row[2])):
				if row[2][j] == '*':
					#print(line_count)
					add = False
					break;
			if add == True:
				if(len(row)>=4):
					raw_primary.append( row[2] )
					raw_secondary.append( row[4] )

			line_count += 1
	print(f'Processed {line_count} lines.')

Processed 10001 lines.


In [3]:
for i, val in enumerate(raw_primary[0:20]):
  print(str(i) + " " + val)


0 EDL
1 KCK
2 KAK
3 KFK
4 KAK
5 KMK
6 KHK
7 KIK
8 KAK
9 KGK
10 KGK
11 KFK
12 KPK
13 KAK
14 KDK
15 KSK
16 KTK
17 KYK
18 KAK
19 KNK


In [4]:
for i in range(len(raw_secondary)):
	len1 = len(raw_secondary[i])
	len2 = len(raw_primary[i])
	if(len1 != len2):
	  print(i," ", raw_secondary[i]," ",raw_primary[i])

In [5]:
def split(sequence):
  return [char for char in sequence]


In [6]:
split_primary = []
split_secondary = []
for i in range(len(raw_primary)):
  split_primary.append(split(raw_primary[i]))
  split_secondary.append(split(raw_secondary[i]))

In [7]:
def orthogonal_primary(arg):
  switch = {
  'A' : np.array([1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]),
  'C' : np.array([0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]),
  'E' : np.array([0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]),
  'D' : np.array([0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]),
  'G' : np.array([0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]),
  'F' : np.array([0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0]),
  'I' : np.array([0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0]),
  'H' : np.array([0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0]),
  'K' : np.array([0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0]),
  'M' : np.array([0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0]),
  'L' : np.array([0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0]),
  'N' : np.array([0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0]),
  'Q' : np.array([0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0]),
  'P' : np.array([0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0]),
  'S' : np.array([0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0]),
  'R' : np.array([0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0]),
  'T' : np.array([0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0]),
  'W' : np.array([0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0]),
  'V' : np.array([0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0]),
  'Y' : np.array([0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1])
  }
  return switch.get(arg)


In [8]:
def orthogonal_secondary(arg):
  switch = {
  'H' : 0,
  'C' : 1,
  'E' : 2
  }
  return switch.get(arg)

In [9]:
for i in range(len(split_primary)):
  seq = split_primary[i]
  for j in range(len(seq)):
    seq[j] = orthogonal_primary(seq[j])

In [10]:
for i in range(len(split_secondary)):
  seq2 = split_secondary[i]
  for j in range(len(seq2)):
    seq2[j] = orthogonal_secondary(seq2[j])


In [11]:
def target(lis):
  Y = []
  print(len(lis))
  for i in range(len(lis)):
    for j in range(len(lis[i])):
      Y.append(lis[i][j])
  return Y


In [12]:
y_label = target(split_secondary)

7366


In [13]:
y_label = np.array(y_label)

In [14]:
def window_padding_data(size, sequence):
	num = int(size/2)
	zeros = np.array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

	for i in range(len(sequence)):
		for j in range(num):
			sequence[i].append(zeros)
			sequence[i].insert(0, zeros)
	X = []
	temp = []
	for k in range(len(sequence)):
		for l in range(len( sequence[k])-(size-1) ):
			temp = sequence[k][l:l+size]
			X.append(temp)
			temp = []
	return X

In [15]:
X = window_padding_data(17,split_primary)


In [16]:
X = np.array(X)

In [17]:
X.shape

(82487, 17, 20)

In [18]:
p=X.shape
X = X.reshape(p[0], 17*20)

In [19]:
X_train, X_test, y_train, y_test = train_test_split(X, y_label, test_size = 0.30)

In [20]:
svc = SVC(kernel='rbf', gamma = 0.1, C=2.5)
svc.fit(X_train, y_train)
y_pred = svc.predict(X_test)
y_true = y_test
print(classification_report(y_true,y_pred))

              precision    recall  f1-score   support

           0       0.87      0.84      0.85      3758
           1       0.93      0.96      0.94     17870
           2       0.89      0.77      0.82      3119

    accuracy                           0.92     24747
   macro avg       0.90      0.86      0.87     24747
weighted avg       0.92      0.92      0.91     24747



In [21]:
svc = SVC(kernel='rbf', gamma = 0.1, C=3.0)
svc.fit(X_train, y_train)
y_pred = svc.predict(X_test)
y_true = y_test
print(classification_report(y_true,y_pred))

              precision    recall  f1-score   support

           0       0.87      0.84      0.85      3758
           1       0.93      0.96      0.94     17870
           2       0.89      0.77      0.83      3119

    accuracy                           0.92     24747
   macro avg       0.90      0.86      0.87     24747
weighted avg       0.92      0.92      0.92     24747



In [22]:
svc = SVC(kernel='rbf', gamma = 0.1, C=3.5)
svc.fit(X_train, y_train)
y_pred = svc.predict(X_test)
y_true = y_test
print(classification_report(y_true,y_pred))

              precision    recall  f1-score   support

           0       0.86      0.84      0.85      3758
           1       0.93      0.96      0.94     17870
           2       0.89      0.78      0.83      3119

    accuracy                           0.92     24747
   macro avg       0.90      0.86      0.88     24747
weighted avg       0.92      0.92      0.92     24747



In [23]:
svc = SVC(kernel='rbf', gamma = 0.1, C=4.0)
svc.fit(X_train, y_train)
y_pred = svc.predict(X_test)
y_true = y_test
print(classification_report(y_true,y_pred))

              precision    recall  f1-score   support

           0       0.87      0.84      0.85      3758
           1       0.93      0.96      0.94     17870
           2       0.89      0.78      0.83      3119

    accuracy                           0.92     24747
   macro avg       0.89      0.86      0.88     24747
weighted avg       0.92      0.92      0.92     24747

